<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->

# **4. Classificação: Knn**
---



# Caso: **Predição de Diagnósticos a partir de Dados de Imagens**


https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)

Os dados acima são características computadas a partir de uma imagem digitalizada por *agulha fina* (PAAF) de uma massa mamária. Eles descrevem características dos núcleos celulares presentes na imagem e classificam os tumores como malignos ou benignos.

Os dados estão na URL: https://meusite.mackenzie.br/rogerio/TIC/breast-cancer-wisconsin.csv





In [220]:
#@markdown **Set Up Lab** (*run this before continue*)

import pandas                  as pd
import numpy                   as np
import matplotlib.pyplot       as plt
import warnings
import os
import seaborn as sns
warnings.filterwarnings("ignore")

breast = pd.read_csv('https://github.com/Rogerio-mack/Machine-Learning-I/raw/main/data/breast.csv')

from sklearn.model_selection import train_test_split

seed = 1984
X_train, X_test, y_train, y_test = train_test_split(breast.drop(columns=['diagnosis']), breast.diagnosis, test_size=0.05, stratify=breast.diagnosis, random_state=seed)

breast = pd.concat([X_train, y_train],axis=1)
new_breast = X_test

breast.to_csv('breast.csv',index=False)

new_breast['diagnosis'] = '?'
new_breast.to_csv('new_breast.csv',index=False)

y_test.to_csv('new_breast_answers.csv',index=False)


E você empregará os dados já tratados no set up acima, com os dataframes `breast` e `new_breast`. A idea aqui é empregar os dados de `breast` para estimar as classes dos novos casos desconhecidos de `new_breast`.



In [221]:
breast.head()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
411,8911670,18.810,19.98,120.90,1102.0,0.08923,0.05884,0.08020,0.05843,0.1550,...,24.30,129.00,1236.0,0.12430,0.1160,0.2210,0.12940,0.2567,0.05737,M
149,9047,12.940,16.17,83.18,507.6,0.09879,0.08836,0.03296,0.02390,0.1735,...,23.02,89.69,580.9,0.11720,0.1958,0.1810,0.08388,0.3297,0.07834,B
103,859196,9.173,13.86,59.20,260.9,0.07721,0.08751,0.05988,0.02180,0.2341,...,19.23,65.59,310.1,0.09836,0.1678,0.1397,0.05087,0.3282,0.08490,B
260,864496,8.726,15.83,55.84,230.9,0.11500,0.08201,0.04132,0.01924,0.1649,...,19.62,64.48,284.4,0.17240,0.2364,0.2456,0.10500,0.2926,0.10170,B
106,91505,12.540,16.32,81.25,476.3,0.11580,0.10850,0.05928,0.03279,0.1943,...,21.40,86.67,552.0,0.15800,0.1751,0.1889,0.08411,0.3155,0.07538,B


In [222]:
new_breast.head()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
274,862009,13.45,18.30,86.60,555.1,0.10220,0.08165,0.03974,0.02780,0.1638,...,25.94,97.59,699.4,0.13390,0.17510,0.13810,0.07911,0.2678,0.06603,?
107,9110127,18.03,16.85,117.50,990.0,0.08947,0.12320,0.10900,0.06254,0.1720,...,22.02,133.30,1292.0,0.12630,0.26660,0.42900,0.15350,0.2842,0.08225,?
264,90769601,11.13,16.62,70.47,381.1,0.08151,0.03834,0.01369,0.01370,0.1511,...,20.29,74.35,421.1,0.10300,0.06219,0.04580,0.04044,0.2383,0.07083,?
175,909777,10.57,18.32,66.82,340.9,0.08142,0.04462,0.01993,0.01111,0.2372,...,23.31,69.35,366.3,0.09794,0.06542,0.03986,0.02222,0.2699,0.06736,?
215,904689,12.96,18.29,84.18,525.2,0.07351,0.07899,0.04057,0.01883,0.1874,...,24.61,96.31,621.9,0.09329,0.23180,0.16040,0.06608,0.3207,0.07247,?


# Acesse e Explore os dados

Qual o atributo classe? Quantos casos são **'B'enignos** e **'M'alignos**? Existem dados ausentes (Null ou NA)?


In [223]:
breast.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 513 entries, 411 to 386
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       513 non-null    int64  
 1   radius_mean              513 non-null    float64
 2   texture_mean             513 non-null    float64
 3   perimeter_mean           513 non-null    float64
 4   area_mean                513 non-null    float64
 5   smoothness_mean          513 non-null    float64
 6   compactness_mean         513 non-null    float64
 7   concavity_mean           513 non-null    float64
 8   concave points_mean      513 non-null    float64
 9   symmetry_mean            513 non-null    float64
 10  fractal_dimension_mean   513 non-null    float64
 11  radius_se                513 non-null    float64
 12  texture_se               513 non-null    float64
 13  perimeter_se             513 non-null    float64
 14  area_se                 

**O atributo classe é o "diagnosis", ele é o rótulo que desejamos classificar**

In [224]:
breast.describe()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,5.130000e+02,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000,...,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000
mean,3.251159e+07,14.130793,19.307096,91.977544,655.179727,0.096381,0.104185,0.088501,0.048872,0.180777,...,16.260316,25.699571,107.204094,879.016959,0.132334,0.253158,0.270106,0.113929,0.289624,0.083757
std,1.312471e+08,3.522711,4.359203,24.284796,353.804031,0.014351,0.052926,0.079784,0.038734,0.027530,...,4.809117,6.241849,33.550555,568.699863,0.023280,0.157925,0.208334,0.064910,0.062367,0.018241
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.688260e+05,11.690000,16.160000,75.170000,419.800000,0.085880,0.063740,0.028910,0.020310,0.161600,...,12.980000,20.980000,83.900000,513.900000,0.115700,0.144000,0.112500,0.064130,0.250200,0.071230
50%,9.055570e+05,13.430000,18.890000,86.870000,556.700000,0.095940,0.093620,0.063350,0.033900,0.179300,...,14.990000,25.410000,97.960000,688.900000,0.131300,0.211600,0.226000,0.101200,0.281000,0.079870
75%,8.812844e+06,15.750000,21.840000,103.700000,781.000000,0.105400,0.130500,0.130700,0.073400,0.195300,...,18.790000,29.890000,125.000000,1084.000000,0.146500,0.337100,0.379100,0.160700,0.318700,0.091850
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [225]:
breast['diagnosis'].value_counts()

B    322
M    191
Name: diagnosis, dtype: int64

In [226]:
breast.isnull().sum()

id                         0
radius_mean                0
texture_mean               0
perimeter_mean             0
area_mean                  0
smoothness_mean            0
compactness_mean           0
concavity_mean             0
concave points_mean        0
symmetry_mean              0
fractal_dimension_mean     0
radius_se                  0
texture_se                 0
perimeter_se               0
area_se                    0
smoothness_se              0
compactness_se             0
concavity_se               0
concave points_se          0
symmetry_se                0
fractal_dimension_se       0
radius_worst               0
texture_worst              0
perimeter_worst            0
area_worst                 0
smoothness_worst           0
compactness_worst          0
concavity_worst            0
concave points_worst       0
symmetry_worst             0
fractal_dimension_worst    0
diagnosis                  0
dtype: int64

# Preparando os dados

Prepare aqui os dados de entrada `X` e saída `y` do modelo.

Note que há um atributo além do atributo classe que não é preditor (qual?). Remova esse atributo antes de normalizar os dados. Para normalização empregue o `MinMaxScaler()` do scikit-learn (ver no material de teoria exemplo de como aplicar).

In [227]:
from sklearn.preprocessing import MinMaxScaler

#Considerando apenas os atributos preditores
x = breast.drop(['id', 'diagnosis'], axis=1)

#Normalização
scaler = MinMaxScaler()
scaler.fit(x)
x = scaler.transform(x)

#Variável alvo
y = breast['diagnosis']

# Conjunto de Treinamento e Teste

Empregue a função `train_test_split` e separe 0.3 dos dados para teste. Empregue o `random_state=123` para garantir a reprodutibilidade dos resultados e empregue o parâmetro `stratify` para garantir a mesma distribuição das classes nos conjuntos de treinamento e teste.

In [228]:
from sklearn.model_selection import train_test_split

#Separação entre treinamento e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify = y, test_size = 0.3, random_state = 123)

# Treine seu modelo Knn

Treine dois modelos Knn, um com k=3 e outro com k=11. Verifique a acuracidade de cada modelo para o conjunto de teste.





In [229]:
from sklearn.neighbors import KNeighborsClassifier

#Definição do classificador k=3
clf3 = KNeighborsClassifier(3)

#Treinamento
clf3.fit(x_train, y_train)

#Avaliação
y_pred3 = clf3.predict(x_test)

print(clf3.score(x_test, y_test))

0.935064935064935


In [230]:
#Definição do classificador k=11
clf11 = KNeighborsClassifier(11)

#Treinamento
clf11.fit(x_train, y_train)

#Avaliação
y_pred11 = clf11.predict(x_test)

print(clf11.score(x_test, y_test))

0.948051948051948


# Classification Report

Produza o Classification report para o modelo com maior acuracidade obtido acima.

In [231]:
from sklearn.metrics import classification_report

#Report da classificação
print(classification_report(y_test, y_pred11))

              precision    recall  f1-score   support

           B       0.95      0.97      0.96        97
           M       0.95      0.91      0.93        57

    accuracy                           0.95       154
   macro avg       0.95      0.94      0.94       154
weighted avg       0.95      0.95      0.95       154



# Prevendo novos Casos

Empregue o melhor modelo obtido acima para classificar os novos casos de `new_breast`. Quantos casos benignos e malignos você encontra?

**Dica**: verifique que os dados dos novos casos precisam estar no mesmo *formato* dos dados empregados no treinamento. Verifique por exemplo os valores de X_train, X_test e compare com os dados de `new_breast`. Veja que os dados parecem estar muito diferentes agora, por que?

```
# Isto está formatado como código
```



In [232]:
#Verificando se existem valores ausente
new_breast.isnull().sum()

id                         0
radius_mean                0
texture_mean               0
perimeter_mean             0
area_mean                  0
smoothness_mean            0
compactness_mean           0
concavity_mean             0
concave points_mean        0
symmetry_mean              0
fractal_dimension_mean     0
radius_se                  0
texture_se                 0
perimeter_se               0
area_se                    0
smoothness_se              0
compactness_se             0
concavity_se               0
concave points_se          0
symmetry_se                0
fractal_dimension_se       0
radius_worst               0
texture_worst              0
perimeter_worst            0
area_worst                 0
smoothness_worst           0
compactness_worst          0
concavity_worst            0
concave points_worst       0
symmetry_worst             0
fractal_dimension_worst    0
diagnosis                  0
dtype: int64

In [233]:
#Removendo a coluna ID
new_breast = new_breast.drop('id', axis=1)

In [234]:
#Convertendo o caractere ? para valores ausentes
new_breast = new_breast.replace('?', np.nan)

In [235]:
#Realizando normalização dos dados
scaler = MinMaxScaler()
scaler.fit(new_breast)
new_breast_norm = scaler.transform(new_breast)

In [236]:
#removendo a coluna do diagnostico
new_breast_norm = new_breast_norm[:, :-1]

In [237]:
#Definição do classificador k=11
clf = KNeighborsClassifier(11)

#Treinamento
clf.fit(x_train, y_train)

#Fazendo previsão de novos casos
y_pred_new = clf.predict(new_breast_norm)

In [241]:
#Transformando os diagnósticos em um dataframe
df_y_pred_new = pd.DataFrame({'diagnosis_new': y_pred_new})

df_y_pred_new.head()

,diagnosis_new
0,B
1,M
2,B
3,B
4,B


In [242]:
#Quandidade de Benignos e malignos
df_y_pred_new['diagnosis_new'].value_counts()

B    16
M    11
Name: diagnosis_new, dtype: int64

**Portanto, a classificação previu 16 casos benignos e 11 casos malignos**